In [1]:
# Standard packages first, then others in alphabetical order

import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests
import warnings

warnings.filterwarnings('ignore')


### BIG MUDDY RIVER AT AT MURPHYSBORO, IL
Analysis of the 2017 flood event 

![The Big Muddy River at Murphysboro, IL.](https://www.weather.gov/images/pah/StormEvents/2017/Apr%2028-Early%20May%20Flooding/3Herrin_Photo%20by%20tony%20laubach.jpg)

> Image Source: [NWS](https://www.weather.gov/pah/2017Apr28-EarlyMayRiverFlooding)

A strong storm system impacted the region over the last few days of April in 2017. Two rounds of significant rainfall affected various areas. The 1st was from the evening hours of April 28th through the mid morning hours on April 29th. A lull in precipitation occurred during much of the daytime hours on the 29th, before the 2nd round of heavy rain moved in from the west during the evening hours, lasting through the morning hours on the 30th. The heaviest rainfall amounts were confined to portions of southeast Missouri into southern Illinois and southwest Indiana. A swath of 6 to 10 inches, with a few isolated higher amounts, was observed from Van Buren, MO to Carbondale and Carmi, IL and to Stendal, IN [(NWS)](https://www.weather.gov/pah/2017Apr28-EarlyMayRiverFlooding).

In [2]:
sg_lat = 37.7583333
sg_lon = -89.3277778

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Stream Gage on the Big Muddy River "
              "near Murphysboro, IL"
              ).add_to(m)

# Display the map
m

USGS streamflow URL: https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05599490&legacy=&referred_module=sw&period=&begin_date=1930-10-01&end_date=2022-09-30

In [3]:
nwis_url = (
    "https://waterdata.usgs.gov/nwis/"
    "dv?cb_00060=on"
    "&format=rdb"
    "&site_no=05599490"
    "&legacy="
    "&referred_module=sw"
    "&period="
    "&begin_date=1930-10-01"
    "&end_date=2022-09-30")

# Send an HTTP GET request to the URL
nwis_response = requests.get(nwis_url)
nwis_response.raise_for_status()

nwis_response

<Response [200]>

In [4]:
murp_q_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment="#",
    delimiter="\t",
    skiprows=[27, 28],
    names=["agency_cd", "site_no", "datetime", "streamflow_cfs", "code"],
    index_col="datetime",
    parse_dates=True,
)

In [5]:
# Subset Murphysboro data records for water years from 2016 oct to 2017 sept
murp_flood_df = murp_q_df['2016-10':'2017-09']

In [6]:
# Alternative plotting: omaha_flood_df.hvplot(y='streamflow_cfs')

murp_flood_2017 = murp_flood_df.streamflow_cfs.hvplot(
    title = 'Big Muddy river near Murphysboro, IL 2016 - 2017 streamflows',
    xlabel='year',
    ylabel='cfs'
)

murp_flood_2017

:Curve   [datetime]   (streamflow_cfs)

In [7]:
# Plot the entire record
murp_cfs_all = murp_q_df.streamflow_cfs.hvplot(
    title = 'Big Muddy river near Murphysboro, IL 1930 - 2022 streamflows',
    xlabel='year',
    ylabel='cfs'
)

murp_cfs_all

:Curve   [datetime]   (streamflow_cfs)

In [8]:
#Resample the streamflow_cfs column as year start maximum values
murp_ann_max_q_df = murp_q_df[['streamflow_cfs']].resample('AS').max()
murp_ann_max_q_df

# Plot resampled data
murp_cfs_ann_max = murp_ann_max_q_df.hvplot(
    title = 'Big Muddy river near Murphysboro, IL 1930 - 2022 '
    'annual max streamflows',
    xlabel='year',
    ylabel='cfs'
)

murp_cfs_ann_max

:Curve   [datetime]   (streamflow_cfs)

In [9]:
# Create a new dataframe for the subset years
murp_ann_max_q_post_df = murp_ann_max_q_df.copy()

# Calculate exceedance probabilities
murp_ann_max_q_post_df['exceed_prob'] = (
    murp_ann_max_q_post_df
    .rank(ascending=False)
    / len(murp_ann_max_q_post_df))

# Calculate return periods
murp_ann_max_q_post_df['return_period'] = (
    1 / murp_ann_max_q_post_df.exceed_prob)

murp_ann_max_q_post_df

# Plot the return periods
murp_return_p = murp_ann_max_q_post_df.return_period.hvplot(
    title = 'Big Muddy river near Murphysboro, IL return periods',
    xlabel='year',
    ylabel='return period'
)

murp_return_p

:Curve   [datetime]   (return_period)

 #### The 2017 event was a relatively frequent flood event in Murphysboro, Illinois. The return period is about 14 years for at least this magnitude floods. I was interested in learning more about the 2017 water year becasue I moved to the nearby town, Carbondale, IL, during this time. Carbondale also had flooded roads due to the excessive rainfall that preceded the floods. Now that I analyzed the available data, the 2011 93-year-return-period event stands out as the major flood in the area.

In [10]:
%%capture
%%bash
jupyter nbconvert time-series-flooding.ipynb --to html --no-input